In [ ]:
import os

import geopandas
import numpy
import pandas
from likeness_vitals.vitals import get_censusapikey

from livelike import acs, make_supertracts, select_constraints
from livelike.config import constraints, up_base_constraints_selection

In [ ]:
censusapikey = get_censusapikey("../")

In [ ]:
# # P-MEDM constraints – base
year = 2019
constraints = constraints.loc[
    (constraints.begin_year <= year) & (constraints.end_year >= year)
]
constraints = select_constraints(constraints, up_base_constraints_selection)

In [ ]:
pup_louvain = acs.puma(
    "4701603",
    constraints.copy(),
    target_zone="trt",
    keep_geo=True,
    censusapikey=censusapikey,
    make_super_trt_method="louvain",
)

In [ ]:
# not sure how this was made
# pup_louvain_few_constrs_universe_constrs = None

In [ ]:
sf_data = pup_louvain.geo.copy()
sf_data["population"] = range(1, sf_data.shape[0] + 1)
sf_data = sf_data.drop(columns=[c for c in sf_data.columns if c != "population"])
# sf_data

In [ ]:
pup_louvain_few_constrt_universe_constrs = pup_louvain.geo.copy().drop(
    columns="super_trt"
)
pup_louvain_few_constrt_universe_constrs["super_trt"] = make_supertracts(
    pup_louvain_few_constrt_universe_constrs,
    sf_data,
    exclude_universe_constraints=False,
    method="louvain",
)

In [ ]:
gpkg = "../livelike/tests/super_tract_expected/super_tract_expected.gpkg"

In [ ]:
# the temp file needs to be zipped and copied to `tests/super_tract_expected`
pup_louvain.geo.to_file(gpkg, layer="louvain")
pup_louvain_few_constrt_universe_constrs.to_file(
    gpkg, layer="louvain_few_constrs_universe_constrs"
)